# КР2

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [2]:
data = pd.read_csv("house_price_regression_dataset.csv")

X = data.drop(columns=["House_Price"])
y = data["House_Price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
X_train.reset_index(inplace=True)
X_test.reset_index(inplace=True)
y_train = y_train.reset_index()["House_Price"]
y_test = y_test.reset_index()["House_Price"]

## Бустинг. - 6 баллов
В существующий код бустинга добавьте возможность ранней остановки обучения. 
должны быть учтены:
1) Наличие валидационного датасета (либо разделение должно быть внутри класса, либо вне его, а в обучении новый набор будет подаваться отдельной парой)
2) Кастомная метрика или лосс для оствновки. Должна передаваться в виде доп. параметра. Дефолт - лосс функция для расчета градиента.
3) Укажите, сколько должно пройти итераций для ранней остановки. 
4) После обучения должно вернуться лучшее состояние модели по валидационной выборке, а не то, которое было достинуто при остановке обучения. 

Для обучения используйте тот же датасет, что использовался на 8 семинаре (house_price_regression_dataset).
1 и 3 пункты обязательны - 3 балла. 2 пункт - 1 балл (при недефолтной реализации). 4 пункт - 2 балла.

In [3]:
class MyGradientRegressor:
    def __init__(self, n_estimators: int = 300, max_depth: int = 3, lr: float = 0.1):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.lr = lr
        self.estimators = []  

    def fit(self, X, y, eval_metric=None, early_stopping_rounds: int = 10, validation_fraction: float = 0.2):
        X = np.array(X)
        y = np.array(y)
        
        # разбиение на трен и вал
        n_samples = X.shape[0]
        indices = np.arange(n_samples)
        np.random.shuffle(indices)
        split_idx = int(n_samples * (1 - validation_fraction))
        train_idx = indices[:split_idx]
        val_idx = indices[split_idx:]
        X_train, y_train = X[train_idx], y[train_idx]
        X_val, y_val = X[val_idx], y[val_idx]
        
        # дефолт метрика
        if eval_metric is None:
            eval_metric = lambda y_true, y_pred: np.mean((y_true - y_pred) ** 2)  
        
        self.estimators = []  
        predictions = np.zeros_like(y_train, dtype=float)  
        val_predictions = np.zeros_like(y_val, dtype=float)  
        best_score = float('inf')
        best_estimators = None
        rounds_without_improve = 0

        for i in range(self.n_estimators):
            new_model = DecisionTreeRegressor(max_depth=self.max_depth)
            new_target = -2 * (predictions - y_train)
            new_model.fit(X_train, new_target)
            self.estimators.append(new_model)
            predictions += self.lr * new_model.predict(X_train)
            val_predictions += self.lr * new_model.predict(X_val)
            
            current_score = eval_metric(y_val, val_predictions)
            if current_score < best_score:
                best_score = current_score
                best_estimators = self.estimators.copy()  
                rounds_without_improve = 0
            else:
                rounds_without_improve += 1
            
            # остановка
            if rounds_without_improve >= early_stopping_rounds:
                print(f"Остановка на итерации {i+1} с метрикой на валидации: {best_score}")
                self.estimators = best_estimators  
                return
        
        print(f"Обучение завершено без остановки с метрикой на валидации: {best_score}")

    def predict(self, X_test):
        X_test = np.array(X_test)
        curr_pred = 0
        for est in self.estimators:
            curr_pred += self.lr * est.predict(X_test)
        return curr_pred

In [4]:
my_model = MyGradientRegressor(n_estimators=300, max_depth=3, lr=0.1)
my_model.fit(X_train, y_train, eval_metric=None, early_stopping_rounds=10, validation_fraction=0.25)
pred = my_model.predict(X_test)
print(mean_absolute_error(y_test, pred))

Остановка на итерации 97 с метрикой на валидации: 291440680.66494006
14134.788195036923


## Стекинг - 4 балла
В текущей реализации в качестве признаков для метамодели используются предсказания базовых моделей.
Ваша задача добавить возможность дополнительно учитывать исходные данные в качестве признаков (гиперпараметр). 
Метапризнаки как доп. фичи к основным.
При этом на основные признаки добавляется воможность расчета полиномиальных признаков (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)

Для тестирования используйте тот же датасет

In [5]:
class Stacking:
    def __init__(self, estimators, meta_estimator, folds=5, include_original_features=False, poly_degree=1):
        self.estimators = estimators
        self.meta_estimator = meta_estimator
        self.folds = folds
        self.include_original_features = include_original_features  
        self.meta_train = []
        self.poly_degree = poly_degree
        
        # если используем исходные признаки 
        if self.include_original_features and self.poly_degree > 1:
            self.poly_transformer = PolynomialFeatures(degree=self.poly_degree, include_bias=False)
        else:
            self.poly_transformer = None
        
    def _fit_estimator(self, estimator, X_train, y_train):
        kf = KFold(n_splits=self.folds, shuffle=True)
        train_fold_indices = []
        test_fold_indices = []
        test_fold_predicts = []

        for train_idx, test_idx in kf.split(X_train):
            train_fold_indices.extend(train_idx)
            test_fold_indices.extend(test_idx)

            estimator.fit(X_train[train_idx], y_train[train_idx])
            test_fold_predicts.extend(estimator.predict(X_train[test_idx]))

        estimator.fit(X_train, y_train)
        self.meta_train.append(np.array(test_fold_predicts)[np.argsort(test_fold_indices)])

    def fit(self, X_train, y_train):
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        self.meta_train = []

        for estimator in self.estimators:
            self._fit_estimator(estimator, X_train, y_train)

        self.meta_train = np.array(self.meta_train).transpose()  

        # если используем исходные признаки 
        if self.include_original_features:
            if self.poly_transformer is not None:
                X_transformed = self.poly_transformer.fit_transform(X_train)
            else:
                X_transformed = X_train
            self.meta_train = np.hstack([self.meta_train, X_transformed])
        
        self.meta_estimator.fit(self.meta_train, y_train)

    def predict(self, X_test):
        X_test = np.array(X_test)
        meta_features = np.array([estimator.predict(X_test) for estimator in self.estimators]).transpose()
        
        # если используем исходные признаки 
        if self.include_original_features:
            if self.poly_transformer is not None:
                X_transformed = self.poly_transformer.transform(X_test)
            else:
                X_transformed = X_test
            meta_features = np.hstack([meta_features, X_transformed])
        
        return self.meta_estimator.predict(meta_features)


In [6]:
model = Stacking(
    estimators=[
        LinearRegression(),
        DecisionTreeRegressor(max_depth=3),
        DecisionTreeRegressor(max_depth=3),
    ],
    meta_estimator=LinearRegression(), include_original_features=True, poly_degree=1)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(mean_absolute_error(y_test, y_pred))

8299.40173212301
